# Explore data augmentations

This notebook will be used to test different data augmentation strategies. In your config file, you can modify the augmentations by adding `data_augmentation_options`. For example, I would like to add a random RGB to gray transform with a probability of 0.2:
```
data_augmentation_options {
    random_rgb_to_gray {
    probability: 0.2
    }
```

A complete list of augmentation is provided the [`preprocessor.proto` file of the Tf Object Detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto).

### Helper functions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
import albumentations as A
import tensorflow as tf
import random

In [ ]:
def recenter_image(image):
    # ssd preprocessing
    image += [123.68, 116.779, 103.939]
    return image

def display_instances(image, bboxes, classes):
    image = recenter_image(image)
    w, h, _ = image.shape
    # resize the bboxes
    bboxes[:, [0, 2]] *= w
    bboxes[:, [1, 3]] *= h
    delta = round(random.uniform(-0.2, 0.2),2)
    trans_image = tf.image.adjust_brightness(image / 255, delta) * 255

    f, ax = plt.subplots(1,2, figsize=(10, 10))
    ax[0].imshow(image.astype(np.uint8))
    ax[1].imshow(trans_image.astype(np.uint8))

    for bb, cl in zip(bboxes, classes):
        y1, x1, y2, x2 = bb
        rec = Rectangle((x1, y1), x2-x1, y2-y1, facecolor='none', edgecolor='r', linewidth=2)
        ax[0].add_patch(rec)
        ax[1].add_patch(rec)

    ax[0].set_title('the original image')
    ax[1].set_title('the transformed image with brightness delta '+str(delta))
    plt.show()

def display_batch(batch):
    # get images, bboxes and classes
    batched_images = batch[0]['image'].numpy()
    batched_bboxes = batch[1]['groundtruth_boxes'].numpy()
    batched_classes = batch[1]['groundtruth_classes'].numpy()
    num_bboxes = batch[1]['num_groundtruth_boxes'].numpy()
    batch_size = batched_images.shape[0]
    for idx in range(batch_size):
        display_instances(batched_images[idx, ...], 
                          batched_bboxes[idx, :num_bboxes[idx], :],
                          batched_classes[idx, ...])


### Display augmentations
Simulate different weather, Only different light and dark conditions are simulated here

In [ ]:
from utils import get_train_input

In [ ]:
train_dataset = get_train_input("./training/reference/pipeline_new.config")

In [ ]:
%matplotlib inline

for batch in train_dataset.take(1):
    display_batch(batch)